In [1]:
import re

In [2]:
with open("input.txt", "r") as infile: 
    contents = infile.read().strip().split("\n")

contents[0:2] + contents[-1:]

['Step G must be finished before step X can begin.',
 'Step X must be finished before step B can begin.',
 'Step T must be finished before step E can begin.']

In [3]:
steps = list()

for line in contents: 
    match = re.search('^Step ([A-Z]) must be finished before step ([A-Z]) can begin.$', line)
    if match:
        steps.append((match.group(1),match.group(2)))

steps[0:2]

[('G', 'X'), ('X', 'B')]

In [4]:
letters = list(set([x[0] for x in steps]).union(set([x[1] for x in steps])))
len(letters)

26

In [5]:
def requirements(letter): 
    letters = list()
    for step in steps: 
        if step[1] == letter: 
            letters.append(step[0])
    return letters

requirements("E")

['S', 'L', 'I', 'A', 'C', 'P', 'T']

In [6]:
def execute(letters):
    done, answer = set(), list()
    while len(done) != len(letters):
        candidates = list()
        for letter in letters: 
            if letter in done: 
                continue
            if len(set(requirements(letter)).difference(done)) == 0: 
                candidates.append(letter)
        for i, letter in enumerate(sorted(candidates)):
            done.add(letter)
            answer.append(letter)
            break
            
    return "".join(answer)

execute(letters)

'GDHOSUXACIMRTPWNYJLEQFVZBK'

## Part 2

In [7]:
FIXEDCOST = 60

def duration(letter):
    return FIXEDCOST + "ABCDEFGHIJKLMNOPQRSTUVWXYZ".index(letter) + 1

duration("C")

63

In [8]:
WORKERS = 5

def schedule(letters):
    #time to tick
    time = 0 
    
    #done to contain letters that are finished
    done = set()
    
    #schedule to map worker to a letter 
    schedule  = dict()
    
    #find an available worker or False
    def available(time): 
        for worker in range(WORKERS): 
            if (time, worker) not in schedule: 
                return worker
        return -1
    
    #assign a letter to a worker
    def assign(time, worker, letter):
        for t in range(time, time + duration(letter)):
            schedule[t, worker] = letter
        return
    
    def isfinished(time, letter):
        for worker in range(WORKERS):
            if schedule.get((time, worker)) == letter: 
                return False
        return True
    
    while len(done) != len(letters): 
        candidates = list()
        for letter in letters: 
            if letter in done: 
                continue
            reqs = requirements(letter)
            if len(set(reqs).difference(done)) == 0: 
                candidates.append((letter, reqs))
        for letter, reqs in candidates:
            if all([isfinished(time, req) for req in reqs]):
                worker = available(time)
                if worker != -1:
                    assign(time, worker, letter)
                    done.add(letter)
        time += 1
    
    return schedule

plan = schedule(letters)

In [9]:
max([x[0] for x in plan.keys()])+1

1024